# Interfejsy w Pythonie

**Interfejs** = kontrakt specyfikujący, jakie metody klasa powinna dostarczyć.

**Cel**: Tworzenie **luźnych powiązań** (loose coupling) - zależność od abstrakcji, nie od konkretnych klas.

## Po co interfejsy?

**Metafora - Restauracja i szef kuchni**:

Otwierasz restaurację. Potrzebujesz szefa kuchni.

**Nie zależy Ci na konkretnej osobie** (Jan Kowalski), ale na **umiejętnościach**:
- Gotowanie
- Zarządzanie zespołem
- Planowanie menu

**Interfejs "Szef"** = lista wymaganych umiejętności.

Każdy, kto spełnia te wymagania, może być szefem. To **luźne powiązanie** - nie jesteś zależny od jednej osoby.

---

**W kodzie**: Interfejs określa metody, klasy implementują je po swojemu.

## Podstawowy interfejs

In [1]:
from abc import ABC, abstractmethod

class MyInterface(ABC):
    """Interfejs - określa kontrakt"""
    
    @abstractmethod
    def do_something(self, value):
        """Każda klasa musi zaimplementować tę metodę"""
        pass
    
    @property
    @abstractmethod
    def some_property(self):
        """Każda klasa musi mieć tę property"""
        pass

## Implementacja interfejsu

In [2]:
class MyClass(MyInterface):
    def __init__(self, value=10):
        self._my_prop = value
    
    def do_something(self, value):
        self._my_prop *= 2 + value
    
    @property
    def some_property(self):
        return self._my_prop

# Test
obj = MyClass()
obj.do_something(3)
print(obj.some_property)

50


## Co się stanie, jeśli nie zaimplementujesz metody?

In [3]:
class BadClass(MyInterface):
    pass  # Brak implementacji

# bad = BadClass()  # TypeError: Can't instantiate abstract class

**Interfejs wymusza implementację** - nie możesz stworzyć instancji bez zaimplementowania wszystkich metod abstrakcyjnych.

## Przykład: Kalkulator podatków

Obliczanie podatków jest złożone. Zasady zmieniają się co roku.

**Chcemy**: 
- Różne kalkulatory dla różnych lat (2023, 2024)
- Wspólny interfejs - każdy ma metodę `calculate_tax()`

In [4]:
class TaxCalculator(ABC):
    """Interfejs kalkulatora podatków"""
    
    @abstractmethod
    def calculate_tax(self, income: float) -> float:
        """Oblicza podatek na podstawie dochodu"""
        pass

## Implementacje dla różnych lat

In [5]:
class TaxCalculator2023(TaxCalculator):
    def calculate_tax(self, income: float) -> float:
        # Zasady 2023
        return income * 0.18

class TaxCalculator2024(TaxCalculator):
    def calculate_tax(self, income: float) -> float:
        # Zasady 2024 - inna stawka
        return income * 0.20


Podatek 2023: 1800.0
Podatek 2024: 2000.0


In [6]:
# clietn code
income = 10000
calc_2023 = TaxCalculator2023()
calc_2024 = TaxCalculator2024()

print(f"Podatek 2023: {calc_2023.calculate_tax(income)}")
print(f"Podatek 2024: {calc_2024.calculate_tax(income)}")

Podatek 2023: 1800.0
Podatek 2024: 2000.0


## Programming to an Interface

**Zamiast** pracować z konkretnymi klasami (`TaxCalculator2023`):
```python
calc = TaxCalculator2023()  # Zależność od konkretnej klasy
```

**Pracuj z interfejsem** (`TaxCalculator`):
```python
calc: TaxCalculator = get_calculator()  # Zależność od abstrakcji
```

## Dependency Injection

**Problem**: Jak wybrać kalkulator bez tworzenia silnego powiązania?

**Źle** (silne powiązanie):

In [7]:
class TaxReport:
    def generate(self, income: float):
        calc = TaxCalculator2023()  # Silne powiązanie z 2023
        tax = calc.calculate_tax(income)
        return f"Podatek: {tax}"

report = TaxReport()
print(report.generate(10000))

Podatek: 1800.0


**Problem**: Chcesz użyć kalkulatora 2024? Musisz **edytować** `TaxReport`.

**Dobrze** (Dependency Injection):

In [8]:
class TaxReport:
    def __init__(self, calculator: TaxCalculator):
        """Wstrzykujemy kalkulator przez konstruktor"""
        self.calculator = calculator
    
    def generate(self, income: float):
        tax = self.calculator.calculate_tax(income)
        return f"Podatek: {tax}"


In [9]:
# Klient decyduje, który kalkulator użyć
report_2023 = TaxReport(TaxCalculator2023())
print(report_2023.generate(10000))

report_2024 = TaxReport(TaxCalculator2024())
print(report_2024.generate(10000))

Podatek: 1800.0
Podatek: 2000.0


**Korzyści**:
- `TaxReport` nie zależy od konkretnego kalkulatora
- Zmiana kalkulatora = zmiana argumentu, nie kodu
- Nowy kalkulator (2025) = nowa klasa, **zero zmian** w `TaxReport`

## Dodanie nowej funkcjonalności bez modyfikacji

In [11]:
# Nowy kalkulator 2025 - nowa klasa
class TaxCalculator2025(TaxCalculator):
    def calculate_tax(self, income: float) -> float:
        # Nowe zasady 2025
        if income < 30000:
            return income * 0.15  # Niższa stawka
        else:
            return income * 0.25  # Wyższa stawka


In [12]:
# Użycie - zero zmian w TaxReport
report_2025 = TaxReport(TaxCalculator2025())
print(report_2025.generate(10000))
print(report_2025.generate(50000))

Podatek: 1500.0
Podatek: 12500.0


## Interfejsy a SOLID

**Open/Closed Principle (OCP)**:
- Nowy kalkulator = nowa klasa (`TaxCalculator2025`)
- Zero modyfikacji w `TaxReport` - otwarte na rozszerzenia, zamknięte na modyfikacje

**Liskov Substitution Principle (LSP)**:
- `TaxCalculator2023`, `TaxCalculator2024`, `TaxCalculator2025` mogą zastąpić `TaxCalculator`
- Wszystkie zachowują kontrakt interfejsu

**Dependency Inversion Principle (DIP)**:
- `TaxReport` zależy od abstrakcji (`TaxCalculator`), nie od konkretnych klas
- "Programujemy poprzez interfejs"

## Python i Duck Typing

**W Pythonie** interfejsy nie są wymagane (duck typing):

In [13]:
# Działa bez interfejsu - duck typing
class SimpleTaxCalculator:
    """Nie dziedziczy po TaxCalculator"""
    def calculate_tax(self, income: float) -> float:
        return income * 0.10

report = TaxReport(SimpleTaxCalculator())
print(report.generate(10000))  # Działa - ma calculate_tax()

Podatek: 1000.0


**Ale interfejsy dają**:
- Wymuszenie implementacji (`@abstractmethod`)
- Dokumentację kontraktu
- Wczesne wykrycie błędów (przy definiowaniu klasy, nie runtime)
- Jasną strukturę kodu

**Pythonowy idiom**: Duck typing dla prostych projektów, interfejsy dla dużych.

## Podsumowanie

### Interfejs = Kontrakt + Luźne powiązania

**Technika**:
1. Zdefiniuj interfejs (klasa abstrakcyjna + `@abstractmethod`)
2. Implementuj interfejs w konkretnych klasach
3. Programuj poprzez interfejs (type hint: `calculator: TaxCalculator`)
4. Wstrzykuj implementację (Dependency Injection)

**Korzyści**:
- Luźne powiązania (Low Coupling)
- Łatwa rozbudowa (nowe klasy, nie edycja istniejących)
- Zgodność z SOLID (OCP, LSP, DIP)
- Wymuszenie kontraktu

**Wzorzec**: Programming to an Interface + Dependency Injection